## Training of Mask R-CNN

The training of the algorithm can be done in the terminal. 

```bash
# cd to the MLtools GIT repository
cd projects/bouldering/
python train_net.py --help # if you want to have information about the different inputs

usage: train_net.py [-h] [--config-file FILE] [--resume] [--eval-only] [--num-gpus NUM_GPUS] [--num-machines NUM_MACHINES] [--machine-rank MACHINE_RANK] [--aug-path FILE]
                    [--min-area-npixels MIN_AREA_NPIXELS] [--optimizer-name OPTIMIZER_NAME] [--scheduler-mode SCHEDULER_MODE] [--dist-url DIST_URL]
                    ...

positional arguments:
  opts                  Modify config options at the end of the command. For Yacs configs, use space-separated "PATH.KEY VALUE" pairs. For python-based LazyConfig, use "path.key=value".

optional arguments:
  -h, --help            show this help message and exit
  --config-file FILE    path to config file
  --resume              Whether to attempt to resume from the checkpoint directory. See documentation of `DefaultTrainer.resume_or_load()` for what it means.
  --eval-only           perform evaluation only
  --num-gpus NUM_GPUS   number of gpus *per machine*
  --num-machines NUM_MACHINES
                        total number of machines
  --machine-rank MACHINE_RANK
                        the rank of this machine (unique per machine)
  --aug-path FILE       path to augmentation file
  --min-area-npixels MIN_AREA_NPIXELS
                        threshold for filtering masks smaller than X pixels
  --optimizer-name OPTIMIZER_NAME
                        optimizer name (SGD, ADAM, ADAMW)
  --scheduler-mode SCHEDULER_MODE
                        mode of cycling in CyclingLR
  --dist-url DIST_URL   initialization URL for pytorch distributed backend. See https://pytorch.org/docs/stable/distributed.html for details.

```